### **Step 1: Importing necesaary libraries**

In [ ]:
import os
import re
import ast
import difflib
import pandas as pd
from collections import Counter

### **Step 2: Load Extracted Code Files**

In [ ]:
final_code_path = "final_clean_code.py"
framewise_code_path = "extracted_code.txt"

def load_file(filepath):
    """Reads a file and returns its content."""
    with open(filepath, "r") as f:
        return f.read()

final_code = load_file(final_code_path)
framewise_code = load_file(framewise_code_path)

### **Step 3: Analyzing Code Complexity**

In [ ]:
def analyze_code_complexity(code):
    """Analyzes Python code to determine its complexity based on AST parsing."""
    try:
        tree = ast.parse(code)
        stats = Counter(
            node.__class__.__name__ for node in ast.walk(tree)
        )
        return stats
    except SyntaxError:
        return {}

code_complexity = analyze_code_complexity(final_code)
print("Code Complexity Analysis:", code_complexity)

### **Step 4: Measuring Keystroke Density & Speed**

In [ ]:
def count_keystrokes(framewise_text):
    """Counts total keystrokes by summing characters in each extracted frame."""
    return sum(len(line) for line in framewise_text.split("\n"))

total_keystrokes = count_keystrokes(framewise_code)
total_frames = framewise_code.count("### frame_")  # Counting extracted frames
keystrokes_per_frame = total_keystrokes / max(total_frames, 1)  # Avoid division by zero
print(f"Keystroke Density: {keystrokes_per_frame:.2f} characters per frame")


### **Step 5: Identifying Debugging Patterns**

In [ ]:
def detect_debugging_patterns(framewise_text):
    """Identifies occurrences of debugging patterns like print statements and error corrections."""
    debug_keywords = ["print", "debug", "traceback", "error", "exception"]
    occurrences = sum(framewise_text.lower().count(keyword) for keyword in debug_keywords)
    return occurrences

debugging_occurrences = detect_debugging_patterns(framewise_code)
print(f"Debugging Actions Count: {debugging_occurrences}")

### **Step 6: Detecting Code Evolution Across Frames**

In [ ]:
def detect_code_evolution(framewise_text):
    """Compares sequential frames to detect how code evolves over time."""
    frames = re.split(r"### frame_\d+\.jpg ###", framewise_text)
    changes = []
    for i in range(len(frames) - 1):
        diff = difflib.unified_diff(frames[i].split("\n"), frames[i+1].split("\n"))
        change_count = sum(1 for line in diff if line.startswith("+") or line.startswith("-"))
        changes.append(change_count)
    return changes

evolution_changes = detect_code_evolution(framewise_code)
avg_changes = sum(evolution_changes) / max(len(evolution_changes), 1)
print(f"Average Code Changes per Frame: {avg_changes:.2f}")

### **Step 7: Saving Results to CSV**

In [ ]:
results = {
    "Metric": ["Function Usage", "Keystroke Density", "Debugging Actions", "Code Evolution Changes"],
    "Value": [code_complexity.get("FunctionDef", 0), keystrokes_per_frame, debugging_occurrences, avg_changes]
}
pd.DataFrame(results).to_csv("analysis_results.csv", index=False)
print("Analysis Complete! Results saved to 'analysis_results.csv'")